In [1]:
import tensorflow as tf

In [39]:
class Convoluion_layer(tf.keras.layers.Layer):
    def __init__(self,conv_filter:int, conv_kernel_shape, 
                 conv_strides:int, 
                 padding:int, 
                 pool:bool, 
                 lrn:bool, 
                 pool_kernel = (3,3), 
                 pool_strides = 2):
        super(Convoluion_layer, self).__init__()
        self.conv_filter = conv_filter
        self.conv_kernel_shape = conv_kernel_shape
        self.conv_strides = conv_strides
        self.lrn = lrn
        self.pool = pool
        self.pool_kernel = pool_kernel
        self.pool_strides = pool_strides
        self.padding = padding
        
        if self.padding != 0:
            self.Pad = tf.keras.layers.ZeroPadding2D((self.padding, self.padding))
        self.Conv = tf.keras.layers.Conv2D(filters = self.conv_filter, 
                                           kernel_size = self.conv_kernel_shape, 
                                           activation = 'relu', 
                                           strides = (self.conv_strides, self.conv_strides), 
                                           padding = 'valid')
        if self.pool:
            self.MaxPool = tf.keras.layers.MaxPool2D(pool_size = self.pool_kernel, 
                                                     strides = self.pool_strides)
        
    def call(self, X):
        if self.padding != 0:
            X = self.Pad(X)
        y = self.Conv(X)
        if self.pool:
            y = self.MaxPool(y)
        if self.lrn:
            y = tf.nn.local_response_normalization(y, alpha = 1e-4, beta = .75,
                                                  depth_radius = 2, bias = 2.)
        return y

In [40]:
class Alexnet(tf.keras.models.Model):
    def __init__(self, last_activation:str, n_labels):
        super(Alexnet, self).__init__()
        self.last_activation = last_activation
        self.n_labels = n_labels
        
        self.Conv = tf.keras.Sequential([
            Convoluion_layer(96, (11,11), 4, 0, True, True),
            Convoluion_layer(256, (5,5), 1, 2, True, True),
            Convoluion_layer(384, (3,3), 1, 1, False, False),
            Convoluion_layer(384, (3,3), 1, 1, False, False),
            Convoluion_layer(256, (3,3), 1, 1, True, False)
        ])
        
        self.FC = tf.keras.Sequential([
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(4096, activation = 'relu'),
            tf.keras.layers.Dropout(.5),
            tf.keras.layers.Dense(4096, activation = 'relu'),
            tf.keras.layers.Dropout(.5),
            tf.keras.layers.Dense(self.n_labels, activation = self.last_activation)
        ])
        
    def call(self, X):
        y = self.Conv(X)
        y = self.FC(y)
        return y

In [41]:
a = Alexnet('softmax', 10)

In [42]:
a.build(input_shape = (16, 227,227,3))

In [43]:
a.summary()

Model: "alexnet_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_6 (Sequential)    (16, 6, 6, 256)           3747200   
_________________________________________________________________
sequential_7 (Sequential)    (16, 10)                  54575114  
Total params: 58,322,314
Trainable params: 58,322,314
Non-trainable params: 0
_________________________________________________________________
